# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921691319168                   -0.69    5.8         
  2   -7.926164076730       -2.35       -1.22    1.0    232ms
  3   -7.926837793402       -3.17       -2.37    1.9    285ms
  4   -7.926861496679       -4.63       -3.04    2.6    250ms
  5   -7.926861649080       -6.82       -3.43    1.8    220ms
  6   -7.926861671500       -7.65       -3.84    1.9    222ms
  7   -7.926861680197       -8.06       -4.28    1.2    232ms
  8   -7.926861681752       -8.81       -4.87    1.6    216ms
  9   -7.926861681859       -9.97       -5.23    2.0    230ms
 10   -7.926861681870      -10.96       -5.72    1.5    224ms
 11   -7.926861681872      -11.65       -6.47    1.8    220ms
 12   -7.926861681873      -12.93       -7.58    2.6    252ms
 13   -7.926861681873   +  -14.75       -7.76    3.2    281ms
 14   -7.926861681873      -14.45       -8.70    1.2    207ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921707105250                   -0.69    5.8         
  2   -7.926167855124       -2.35       -1.22    1.0    213ms
  3   -7.926836947128       -3.17       -2.37    1.8    237ms
  4   -7.926861517675       -4.61       -3.02    2.8    263ms
  5   -7.926861638067       -6.92       -3.36    1.9    252ms
  6   -7.926861667936       -7.52       -3.76    1.5    209ms
  7   -7.926861680546       -7.90       -4.37    1.5    210ms
  8   -7.926861681808       -8.90       -4.94    2.0    254ms
  9   -7.926861681854      -10.34       -5.16    2.0    226ms
 10   -7.926861681871      -10.77       -5.93    1.5    211ms
 11   -7.926861681873      -11.92       -6.70    2.1    236ms
 12   -7.926861681873      -13.40       -7.23    2.5    253ms
 13   -7.926861681873   +  -14.75       -8.14    2.1    238ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921697478381                   -0.69    5.9         
  2   -7.926169590761       -2.35       -1.22    1.0    251ms
  3   -7.926841820523       -3.17       -2.37    1.8    230ms
  4   -7.926864939431       -4.64       -3.05    2.8    257ms
  5   -7.926865056485       -6.93       -3.41    1.9    218ms
  6   -7.926865080470       -7.62       -3.79    1.9    244ms
  7   -7.926865091658       -7.95       -4.36    1.2    190ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
